## Model Versioning Design Pattern

In the Model Versioning design pattern, backward compatibility is achieved by deploying a changed model as a microservice with a different REST endpoint. This is a necessary prerequisite for many of the other patterns discussed in this chapter.

In [27]:
import json
import numpy as np
import pandas as pd
import xgboost as xgb
import tensorflow as tf

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from google.cloud import bigquery

## Download and preprocess data

You'll need to authenticate to your Google Cloud to run the BigQuery query below.

In [2]:
from google.colab import auth
auth.authenticate_user()

In the following cell, replace `your-cloud-project` with the name of your GCP project.

In [3]:
# Note: this query may take a few minutes to run
%%bigquery df --project your-cloud-project
SELECT
  arr_delay,
  carrier,
  origin,
  dest,
  dep_delay,
  taxi_out,
  distance
FROM
  `cloud-training-demos.flights.tzcorr`
WHERE
  extract(year from fl_date) = 2015
ORDER BY fl_date ASC
LIMIT 300000

In [4]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [5]:
df.head()

,arr_delay,carrier,origin,dest,dep_delay,taxi_out,distance
288655,4.0,US,PHX,SJC,-1.0,16.0,621.0
45636,-15.0,WN,OKC,STL,1.0,12.0,462.0
247620,-1.0,WN,OAK,LAX,7.0,7.0,337.0
294786,-21.0,UA,PDX,DEN,-8.0,12.0,991.0
282690,-21.0,US,MCI,PHX,-4.0,10.0,1044.0


In [6]:
# Only include origins and destinations that occur frequently in the dataset
df = df[df['origin'].map(df['origin'].value_counts()) > 500]
df = df[df['dest'].map(df['dest'].value_counts()) > 500]

In [7]:
df = pd.get_dummies(df, columns=['carrier', 'origin', 'dest'])

## Model version #1: predict whether or not the flight is > 30 min delayed

In [8]:
# Create a boolean column to indicate whether flight was > 30 mins delayed
df.loc[df['arr_delay'] >= 30, 'arr_delay_bool'] = 1
df.loc[df['arr_delay'] < 30, 'arr_delay_bool'] = 0

In [9]:
df['arr_delay_bool'].value_counts()

0.0    195781
1.0     33962
Name: arr_delay_bool, dtype: int64

In [10]:
classify_model_labels = df['arr_delay_bool']
classify_model_data = df.drop(columns=['arr_delay', 'arr_delay_bool'])

In [11]:
x,y = classify_model_data,classify_model_labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [12]:
model = xgb.XGBRegressor(
    objective='reg:logistic'
)

In [13]:
# Given the dataset size, this may take 1-2 minutes to run
model.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:logistic', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [14]:
y_pred = model.predict(x_test)

In [15]:
acc = accuracy_score(y_test, np.round(y_pred))
print(acc)

0.9651438122431925


In [16]:
# Save the model
model.save_model('model.bst')

### Deploying classification model to AI Platform

Replace `your-cloud-project` below with the name of your cloud project.

In [17]:
# Set your cloud project
PROJECT = 'your-cloud-project'
!gcloud config set project $PROJECT

Updated property [core/project].


In [21]:
BUCKET = PROJECT + '_flight_model_bucket'

In [ ]:
# Create a bucket if you don't have one
# You only need to run this once
!gsutil mb gs://$BUCKET

In [23]:
!gsutil cp 'model.bst' gs://$BUCKET

Copying file://model.bst [Content-Type=application/octet-stream]...
/ [1 files][ 67.4 KiB/ 67.4 KiB]                                                
Operation completed over 1 objects/67.4 KiB.                                     


In [ ]:
# Create the model resource
!gcloud ai-platform models create flight_delay_prediction

Using endpoint [https://ml.googleapis.com/]

Learn more about regional endpoints and see a list of available regions: https://cloud.google.com/ai-platform/prediction/docs/regional-endpoints
Created ml engine model [projects/sara-cloud-ml/models/flight_delay_prediction].


In [24]:
# Create the version
!gcloud ai-platform versions create 'v1' \
  --model 'flight_delay_prediction' \
  --origin gs://$BUCKET \
  --runtime-version=1.15 \
  --framework 'XGBOOST' \
  --python-version=3.7

Using endpoint [https://ml.googleapis.com/]


In [28]:
# Get a prediction on the first example from our test set
!rm input.json
num_examples = 10
with open('input.json', 'a') as f:
  for i in range(num_examples):
    f.write(str(x_test.iloc[i].values.tolist()))
    f.write('\n')

In [29]:
!cat input.json

[5.0, 59.0, 241.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[-3.0, 19.0, 1587.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [32]:
# Make a prediction to the deployed model
!gcloud ai-platform predict --model 'flight_delay_prediction' --version \
  'v1' --json-instances 'input.json'

Using endpoint [https://ml.googleapis.com/]
[0.4868972599506378, 0.004682584665715694, 0.004437300376594067, 0.004618476610630751, 0.01615077443420887, 0.0025173116009682417, 0.06883466243743896, 0.0035972497425973415, 0.008341211825609207, 0.01127849891781807]


In [33]:
# Compare this with actual values
print(y_test.iloc[:5])

140323    0.0
66094     0.0
63096     0.0
192118    0.0
222633    0.0
Name: arr_delay_bool, dtype: float64


## Model version #2: replace XGBoost with TensorFlow

In [36]:
tf_model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=[len(x_train.iloc[0])]),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')                       
])

tf_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
tf_model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
4847/4847 [==============================] - 8s 2ms/step - loss: 0.2069 - accuracy: 0.9468 - val_loss: 0.1038 - val_accuracy: 0.9635
Epoch 2/10
4847/4847 [==============================] - 8s 2ms/step - loss: 0.1114 - accuracy: 0.9609 - val_loss: 0.1003 - val_accuracy: 0.9654
Epoch 3/10
4847/4847 [==============================] - 8s 2ms/step - loss: 0.1042 - accuracy: 0.9630 - val_loss: 0.1198 - val_accuracy: 0.9578
Epoch 4/10
4847/4847 [==============================] - 8s 2ms/step - loss: 0.1006 - accuracy: 0.9640 - val_loss: 0.1009 - val_accuracy: 0.9638
Epoch 5/10
4847/4847 [==============================] - 8s 2ms/step - loss: 0.0993 - accuracy: 0.9643 - val_loss: 0.1046 - val_accuracy: 0.9619
Epoch 6/10
4847/4847 [==============================] - 8s 2ms/step - loss: 0.0971 - accuracy: 0.9654 - val_loss: 0.1015 - val_accuracy: 0.9643
Epoch 7/10
4847/4847 [==============================] - 8s 2ms/step - loss: 0.0968 - accuracy: 0.9652 - val_loss: 0.0989 - val_accuracy:

Note that accuracy will be similar to the XGBoost model. We're just using this to demonstrate how training a model with a different framework could be deployed as a new version.

In [38]:
metrics = tf_model.evaluate(x_test, y_test)
print(metrics)

1795/1795 [==============================] - 2s 1ms/step - loss: 0.0954 - accuracy: 0.9661
[0.09541851282119751, 0.9660840034484863]


Next we'll deploy the updated TF model to AI Platform as a v2.

In [48]:
tf_model_path = 'gs://' + BUCKET + '/tf'

In [ ]:
tf_model.save(tf_model_path, save_format='tf')

In [51]:
!gcloud ai-platform versions create 'v2' \
  --model 'flight_delay_prediction' \
  --origin $tf_model_path \
  --runtime-version=2.1 \
  --framework 'TENSORFLOW' \
  --python-version=3.7

Using endpoint [https://ml.googleapis.com/]


In [ ]:
# Make a prediction to the new version
!gcloud ai-platform predict --model 'flight_delay_prediction' --version \
  'v2' --json-instances 'input.json'

## Alternative: reframe as a regression problem

In this case, you'd likely want to create a new model resource since the response format of your model has changed.

In [ ]:
regression_model_labels = df['arr_delay']
regression_model_data = df.drop(columns=['arr_delay', 'arr_delay_bool'])

In [ ]:
x,y = regression_model_data,regression_model_labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [ ]:
model = xgb.XGBRegressor(
    objective='reg:linear'
)

In [ ]:
# This will take 1-2 minutes to run
model.fit(x_train, y_train)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
for i,val in enumerate(y_pred[:10]):
  print(val)
  print(y_test.iloc[i])
  print()

In [ ]:
model.save_model('model.bst')

In [ ]:
!gsutil cp model.bst gs://$BUCKET/regression/

Copying file://model.bst [Content-Type=application/octet-stream]...
/ [1 files][ 67.8 KiB/ 67.8 KiB]                                                
Operation completed over 1 objects/67.8 KiB.                                     


In [ ]:
!gcloud ai-platform models create 'flights_regression'

In [ ]:
# Create the version
!gcloud ai-platform versions create 'v1' \
  --model 'flights_regression' \
  --origin gs://$BUCKET/regression \
  --runtime-version=1.15 \
  --framework 'XGBOOST' \
  --python-version=3.7

Using endpoint [https://ml.googleapis.com/]


In [ ]:
!gcloud ai-platform predict --model 'flighs_regression' --version \
  'v1' --json-instances 'input.json'

Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License